# Secure Data Disclosure: Server side

This notebook showcases how data owner could set up the server, add make their data available to certain user. It explains the different steps necessary.

## Create a docker volume
The first step is to create a docker volume for mongodb. Docker volumes are persistent storage spaces that are managed by docker and can be mounted in containers. To create the volume use `docker volume create mongodata`. This must be done only once, and we use bind mounts for the server, so no need to create volumes for that.

In [2]:
# In the terminal
#!docker volume create mongodata

mongodata


## Start server
The second step is to start the server. Therefore the config file `configs/example_config.yaml` has to be adapted. The data owner must make sure to set the develop mode to False, specify the database type and ports. For this notebook, we will keep the default and use a mongodb on port 27017. Note: Keep in mind that if the configuration file is modified then the `docker-compose` has to be modified accordingly. This is out of scope for this notebook.

Startup the service by running `docker compose up` in a terminal.

In [6]:
# In the terminal
#!docker compose down

## Current version

In [1]:
# In the terminal
#!docker build --target sdd_server_dev -t sdd_server_dev:latest .

In [2]:
# In the terminal
#!docker run --network test-network -p 80:80 -v /home/pauline/poc/sdd-poc-server/src/:/code/  -v /home/pauline/poc/sdd-poc-server/configs/example_config.yaml:/usr/sdd_poc_server/runtime.yaml  -v /home/pauline/poc/sdd-poc-server/configs/user_database.yaml:/usr/sdd_poc_server/user_database.yaml  -it sdd_server_dev /bin/bash

## Access the database

In [1]:
# MongoDB
MONGODB_CONTAINER_NAME = "mongodb"
MONGODB_PORT = "27017"
DATABASE_NAME = "user_database"

In [2]:
from mongodb_admin import MongoDB_Admin

mongo_admin = MongoDB_Admin(
    f"mongodb://{MONGODB_CONTAINER_NAME}:{MONGODB_PORT}/"
)

In [3]:
mongo_admin.db

Database(MongoClient(host=['mongodb:27017'], document_class=dict, tz_aware=False, connect=True), 'user_database')

For now the database is empty. We will create a collection named `users` and fill it in the next steps.

## Visualise all options
You can visualise all the options offered by the database by running the command `python src/mongodb_admin.py --help`. We will go through through each of them in the rest of the notebook.

In [7]:
!python src/mongodb_admin.py --help

usage: MongoDB administration script for the SDD POC Server
       [-h]
       {add_user,add_user_with_budget,del_user,add_dataset_to_user,del_dataset_to_user,set_budget_field,set_may_query,add_metadata,del_metadata,drop_collection,create_ex_users}
       ...

options:
  -h, --help            show this help message and exit

subcommands:
  {add_user,add_user_with_budget,del_user,add_dataset_to_user,del_dataset_to_user,set_budget_field,set_may_query,add_metadata,del_metadata,drop_collection,create_ex_users}
                        database administration operations
    add_user            add user to users collection
    add_user_with_budget
                        add user to users collection
    del_user            delete user from users collection
    add_dataset_to_user
                        add dataset with initialized budget values for a user
                        in users collection
    del_dataset_to_user
                        delete dataset for user in users collection
  

## Add dataset
The super confidential dataset that we want to add is the penguin dataset from.

In [8]:
DATASET = "PENGUIN" # TODO because nothing yet

## Add metadata

To later perform query on the dataset, some metadata is required. In this secure server the metadata information is expected to be in the same format as [SmartnoiseSQL dictionary format](https://docs.smartnoise.org/sql/metadata.html#dictionary-format), where among other, there is information about all the available columns, their type, bound values (see Smartnoise page for more details).

Finally let's have a look at the  stored metadata:

In [ ]:
!python mongodb_admin.py show_collection --collection metadata

## Add user
Let's add few users.

In [ ]:
!python mongodb_admin.py add_user_with_budget --user "Mrs. Daisy" --dataset "IRIS" --epsilon 10.0 --delta 0.001

In [8]:
!python mongodb_admin.py add_user_with_budget --user "Mr. Coldheart" --dataset "PENGUIN" --epsilon 10.0 --delta 0.001

python: can't open file '/home/pauline/poc/sdd-poc-server/mongodb_admin.py': [Errno 2] No such file or directory


In [9]:
!python mongodb_admin.py add_user_with_budget --user "Dr. Antartica" --dataset "PENGUIN" --epsilon 10.0 --delta 0.001

python: can't open file '/home/pauline/poc/sdd-poc-server/mongodb_admin.py': [Errno 2] No such file or directory


Users must all have different names, otherwise you will have an error and nothing you be done:

In [ ]:
!python mongodb_admin.py add_user_with_budget --user "Dr. Antartica" --dataset "IRIS" --epsilon 10.0 --delta 0.001

If you want to add another dataset access to an existing user, just use the function `add_dataset_to_user` command.

In [ ]:
!python mongodb_admin.py add_dataset_to_user --user "Dr. Antartica" --dataset "IRIS" --epsilon 5.0 --delta 0.005

Alternativelly, you can create user without ataset and then add dataset in another command.

In [ ]:
!python mongodb_admin.py add_user --user "Snugglefluff"
!python mongodb_admin.py add_dataset_to_user --user "Snugglefluff" --dataset "IRIS" --epsilon 5.0 --delta 0.005

Let's see the current state of the database:

In [ ]:
!python mongodb_admin.py show_collection --collection users

Do not hesitate to re-run this call after every other command to ensure that all runs as expected.

## Remove user
You have just heard the the penguin named Coldheart might have malicious intentions and decide to remove his access until an investigation has been made. You can set that he is not allowed to do query anymore.

In [ ]:
!python mongodb_admin.py set_may_query --user "Mr. Coldheart" --value False

Now, he won't be able to do any query (unless you re-run the query with --value True).

A few days have passed and the investigation reveals that he aimed to do unethical research, you can remove the dataset access to him:

In [ ]:
!python mongodb_admin.py del_dataset_to_user --user "Mr. Coldheart" --dataset "PENGUIN"

Or delete him completelly from the codebase

In [ ]:
!python mongodb_admin.py del_user --user "Mr. Coldheart"

Let's see the resulting users:

In [ ]:
!python mongodb_admin.py show_collection --collection users

## Change budget
You also change your ming about the budget allowed to Dr. Antartica and give her a bit more on the penguin dataset.

In [ ]:
!python mongodb_admin.py set_budget_field --user "Dr. Antartica"  --dataset "PENGUIN" --field initial_epsilon --value 15.0

In [ ]:
!python mongodb_admin.py set_budget_field --user "Dr. Antartica"  --dataset "PENGUIN" --field initial_delta --value 0.005

Let's check all our change by looking a the state of the database:

In [ ]:
!python mongodb_admin.py show_collection --collection users

## Stop the server
To tear down the service, use `docker compose down` in your terminal. This will also delete all the containers but the volume will stay in place. 

In [1]:
# In the terminal
!docker compose down